In [10]:
import random
import secrets

import protarrow
from demonstartor_pb2 import SearchRequest

import ptars

In [11]:
pool = ptars.HandlerPool()
handler = pool.get_for_message(SearchRequest.DESCRIPTOR)

In [12]:
MESSAGE_COUNT = 10_000
STRING_SIZE = 10
MIN_INT = -2147483648
MAX_INT = 2147483647

In [13]:
messages = [
    SearchRequest(
        query=secrets.token_urlsafe(random.randint(0, STRING_SIZE)),
        page_number=random.randint(MIN_INT, MAX_INT),
        result_per_page=random.randint(MIN_INT, MAX_INT),
    )
    for _ in range(MESSAGE_COUNT)
]

payloads = [m.SerializeToString() for m in messages]

In [14]:
%%timeit
protarrow.messages_to_record_batch([SearchRequest.FromString(p) for p in payloads], SearchRequest) # noqa E501

7.71 ms ± 148 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
handler.list_to_record_batch(payloads)

7.02 ms ± 20.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
record_batch = handler.list_to_record_batch(payloads)

In [17]:
%%timeit
handler.record_batch_to_array(record_batch)

6.5 ms ± 22.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit 
[m.SerializeToString() for m in protarrow.record_batch_to_messages(record_batch, SearchRequest)] # noqa E501

19.7 ms ± 125 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
